# Regularisation Ridge regression
The following notebook contains pymc models for a Bayesian implementations lasso regression . The model is implemented in fantastic format within the Bayesreg package produced by Makalic & Schmidt (2016). Of which, is expertly explained within great review and methods tutorial paper by Van Erp, Oberski & Mulder (2019). Such a package is very helpful. However, for self pedagogy and modeliing flexibility implementing these models in a differnt PPL is beneficial.

In [244]:
# Import relevant analysis packages.
import pymc as pm
import numpy as np 
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
import pymc.sampling_jax
from sklearn.preprocessing import StandardScaler

In [406]:
# Script functions

# Prediction mean squared error for Bayesian regularized regression model function
# 
def pmse_lm(ppc, ytest):
    # COnvert MCMC sampels into pandas DF
    draws = ppc.to_dataframe()
    # extract only the predicted values
    draws = draws.iloc[:,2:]
    #Generate empty list
    postmeans = []
    # Loop over PPC sampled data set samples and cacculate the mean
    for i in range(np.shape(draws)[1]):
        m = np.mean(draws.iloc[:,i])
        postmeans.append(m)
    
    pmse = np.mean((postmeans-ytest)**2) 
    return pmse
    
def select_p(idata, prob = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    l = []
    for i in prob:
        hdis = pm.hdi(idata, var_names='beta', hdi_prob=i).to_dataframe()
        for i in range(int(np.shape(hdis)[0]/2)):
            x = hdis.beta[i][['lower','higher'][0]] <= 0 and hdis.beta[i][['lower','higher'][1]] >= 0
            l.append(x)

    chunks = [l[x:x+np.shape(hdis)[0]/2] for x in range(0, len(l), np.shape(hdis)[0]/2)]   
    return chunks

In [215]:
# Read dataset from github associated with this notebook.
d = pd.read_csv('https://raw.githubusercontent.com/HPCurtis/Datasets/main/fat.csv', sep=',')
# Copy dataframe for later use
dc = d

# Check for missing values.
if np.sum(np.sum(d.isnull())) >0:
    print('missing values in dataset')
else:
    # Standardise data
    d = pd.DataFrame(StandardScaler().fit_transform(d.iloc[:,1:]))
    
    # Generate regression design matrix
    dm = d.iloc[:, 1:]
    dm.insert(0, '0', np.ones(len(dm)))
    
    #Convert dm to numpy array
    dm = np.asarray(dm)

## Bayesian Ridge regression

In [419]:
with pm.Model() as model:  # model specifications in PyMC are wrapped in a with-statement
    # Priors
    sigma = pm.Uniform("sigma", 0, 1e6)
    lamda = pm.HalfCauchy('lamda', 1)
    beta = pm.Normal("beta", 0, (sigma**2)/lamda, shape = np.shape(dm)[1])

    mu = pm.math.dot(dm, beta)
    # Likelihood
    y = pm.Normal("y", mu = mu, sigma=sigma, observed = d[0].values)

In [420]:
# Fit and sample the model parameters
with model:
    idata = pymc.sampling_jax.sample_numpyro_nuts()
    ppc = pm.sample_posterior_predictive(idata)

Compiling...
Compilation time =  0:00:02.247919
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

Sampling: [y]


Sampling time =  0:00:03.445781
Transforming variables...
Transformation time =  0:00:00.023857


In [421]:
az.summary(idata, hdi_prob = .95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.000,0.001,-0.003,0.003,0.0,0.0,6191.0,2475.0,1.0
beta[1],0.956,0.012,0.932,0.980,0.0,0.0,1548.0,2205.0,1.0
beta[2],-0.026,0.009,-0.044,-0.008,0.0,0.0,2318.0,2801.0,1.0
beta[3],-0.001,0.002,-0.005,0.003,0.0,0.0,4777.0,2961.0,1.0
beta[4],0.034,0.014,0.008,0.062,0.0,0.0,1610.0,2380.0,1.0
beta[5],-0.000,0.002,-0.004,0.004,0.0,0.0,4870.0,3277.0,1.0
beta[6],-0.008,0.006,-0.019,0.004,0.0,0.0,2690.0,2697.0,1.0
beta[7],-0.025,0.010,-0.045,-0.004,0.0,0.0,1575.0,2520.0,1.0
beta[8],0.000,0.003,-0.006,0.006,0.0,0.0,6063.0,3298.0,1.0
beta[9],0.003,0.005,-0.007,0.012,0.0,0.0,3320.0,2559.0,1.0


In [410]:
select_p(idata)[0]

[False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [416]:
select_p(idata, prob= [.1,.2])

[[False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 [True,
  False,
  False,
  True,
  False,
  True,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False]]